In [1]:
import pandas as pd
import numpy as np
import json 
import networkx as nx 
import linkpred

In [9]:
with open('../data_collection/data/tuscany_init.json') as jf:
    data = json.load(jf)
    
len(data)

15863

In [10]:
tuscany = dict()

for restaurant, reviews in data.items():
    if len(reviews)>0 : tuscany[restaurant] = dict()
    else: continue
    for review in reviews : 
        username = review['username']
        date = review['date']
        
        if username not in tuscany[restaurant] : tuscany[restaurant][username] = [date]
        else:                                    tuscany[restaurant][username].append(date)

In [11]:
with open('../data_collection/data/tuscany_alldates.json','w+') as jf:
    json.dump(tuscany,jf)

In [70]:
with open('../data_collection/data/tuscany_alldates.json') as jf:
    tuscany = json.load(jf)

In [71]:
len(tuscany)

15863

In [72]:
tuscanyCovid = dict()
for restaurant, reviews in tuscany.items():
    tuscanyCovid[restaurant] = dict()
    for reviewer,dates in reviews.items() : 
        tuscanyCovid[restaurant][reviewer] = [int(date) for date in dates if int(date) >= 20200000]

In [38]:
deleteRestaurants = []
deleteReviewers = dict()
for restaurant, reviews in tuscanyCovid.items():
    if len(reviews)==0 : deleteRestaurants.append(restaurant)
    else: 
        
        for reviewer,dates in reviews.items() : 
            
            if len(dates)==0: 
                if restaurant in deleteReviewers: deleteReviewers[restaurant].append(reviewer)
                else : deleteReviewers[restaurant] = [reviewer]

In [39]:
for restaurant in deleteRestaurants: tuscanyCovid.pop(restaurant)


In [36]:
for restaurant in deleteReviewers:
    for reviewer in deleteReviewers[restaurant]:
         tuscanyCovid[restaurant].pop(reviewer)

In [38]:
deleteRestaurants = []
deleteReviewers = dict()
for restaurant, reviews in tuscanyCovid.items():
    if len(reviews)==0 : deleteRestaurants.append(restaurant)
    else: 
        
        for reviewer,dates in reviews.items() : 
            
            if len(dates)==0: 
                if restaurant in deleteReviewers: deleteReviewers[restaurant].append(reviewer)
                else : deleteReviewers[restaurant] = [reviewer]

In [39]:
for restaurant in deleteRestaurants: tuscanyCovid.pop(restaurant)


In [80]:
G = nx.MultiDiGraph()

In [75]:
tuscany = tuscanyCovid

In [77]:
with open('../data_collection/data/tuscanyCovid.json','w+') as jf:
    json.dump(tuscany,jf)

In [82]:
for i,reviews_i in tuscany.items():
    if int(i) % 1000 == 0: print(i, end= ' ')
    if len(tuscany[str(i)])==0: continue
    reviewers_i = set(reviews_i.keys())

    for j, reviews_j in list(tuscany.items())[int(i)+1:]:
        if len(tuscany[str(j)])==0: continue

        reviewers_j = set(reviews_j.keys())
       
        intersection = list(reviewers_i.intersection(reviewers_j))
        if len(intersection) > 0:
            for reviewer in intersection:
                dates_i = reviews_i[reviewer]
                dates_j = reviews_j[reviewer]
               
                datepairs = []
                for date_i in dates_i: 
                    for date_j in dates_j:

                        if date_i < date_j:  G.add_edge(i,j, start=date_i,end=date_j)
                        elif date_i > date_j:  G.add_edge(j,i, start=date_j,end=date_i)
                        else: 
                            G.add_edge(j,i, start=date_j,end=date_i)
                            G.add_edge(i,j, start=date_i,end=date_j)

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 

In [84]:
nx.write_graphml(G,'../data/multicoviddigraph.graphml')

In [86]:
len(G.edges)

225902

In [2]:
with open('../data_collection/data/tuscany_alldates.json') as jf:
    tuscany = json.load(jf)

In [3]:
tuscanyCovid = dict()
for restaurant, reviews in tuscany.items():
    tuscanyCovid[restaurant] = dict()
    for reviewer,dates in reviews.items() : 
        tuscanyCovid[restaurant][reviewer] = [int(date) for date in dates if int(date) >= 20200224]

In [5]:
tuscany = tuscanyCovid

In [6]:
tuscanyCovid = dict()
for restaurant, reviews in tuscany.items():
    for reviewer,dates in reviews.items() : 
        if reviewer not in tuscanyCovid: tuscanyCovid[reviewer] = list()
        for date in dates:
            tuscanyCovid[reviewer].append((restaurant,date))

In [7]:
with open('../data_collection/data/tuscanyCovidReviewer.json','w+') as jf:
    json.dump(tuscanyCovid,jf)

In [8]:
tuscany = tuscanyCovid

In [9]:
G = nx.MultiDiGraph()

In [22]:
sum([len(r)-1 for r in tus.values()])

48945

In [23]:
len(tus)

24653

In [4]:
from tqdm.notebook import tqdm

In [24]:
for reviewer, reviews in tqdm(tuscany.items()):
    reviews.sort(key = lambda x: x[1])
    for i,review in enumerate(reviews): 
        restaurant_i, date_i = review
        if len(reviews) > i + 1 :
            restaurant_j, date_j = reviews[i+1]
            G.add_edge(restaurant_i,restaurant_j, start=date_i,end=date_j,user = reviewer)
            if date_i == date_j: G.add_edge(restaurant_j,restaurant_i, start=date_j,end=date_i,user = reviewer)
            
        

  0%|          | 0/1021021 [00:00<?, ?it/s]

In [25]:
len(G.edges)

72098

In [26]:
nx.write_graphml(G,'../data/covidByUser.graphml')

In [114]:
C = nx.read_graphml('../data/covidByUser.graphml')

In [117]:
len(C.edges)

93020

In [28]:
tuscanyCovid = dict()
for reviewer, reviews in tqdm(tuscany.items()):
    if len(reviews) > 1 : tuscanyCovid[reviewer] = {review[1]: list() for review in reviews}
    else: continue
    for review in reviews:
        restaurant, date = review
        tuscanyCovid[reviewer][date].append(restaurant)
    

  0%|          | 0/1021021 [00:00<?, ?it/s]

In [61]:
G = nx.MultiDiGraph()
tuscany = tuscanyCovid

In [50]:

from itertools import product
from itertools import permutations 


[i for i in product(sorted(mydict),2)]

TypeError: 'int' object is not iterable

In [62]:
len(G)

0

In [63]:


from itertools import product
from itertools import permutations


for reviewer, allreviews in tqdm(tuscany.items()):

    allreviews = {k:v for k,v in sorted(allreviews.items())}
    for i, date_i in enumerate(allreviews): 
        reviews_i = allreviews[date_i]
        if len(reviews_i) > 1 :
            newedges = permutations(reviews_i,2)
            for u,v in newedges:
                G.add_edge(u,v, start=date_i,end=date_i,user = reviewer)
             
        
        if len(allreviews) > i + 1 :
            date_j = list(allreviews)[i+1]
            reviews_j = allreviews[date_j]
            newedges = product(reviews_i,reviews_j)
            for u,v in newedges:
                G.add_edge(u,v, start=date_i,end=date_j,user = reviewer)
               
            
        

  0%|          | 0/24653 [00:00<?, ?it/s]

In [65]:
len(G.edges)

111863

In [66]:
nx.write_graphml(G,'../data/covidByUser.graphml')